In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
from pythainlp.tag import tag_provinces
from datetime import datetime
import math


In [ ]:
class spyder:
    def __init__( self ,links,base_url,depth ):
        self.base_url = base_url
        target_links={}
        for i in links:
            target_links[i]=0 
        self.target_links = target_links
        self.depth = depth
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,self.depth,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.check_domain_result = self.check_domain(self.base_url,self.get_crawler())
        return self.check_domain_result
    
    def get_check_not_domain(self):
        self.check_not_domain_result = self.check_not_domain(self.base_url,self.get_crawler())   
        return self.check_not_domain_result
    
    def get_check_ref(self):
        self.check_ref_result = self.check_ref(self.get_check_not_domain(),self.target_links)
        return self.check_ref_result
    
    def get_all(self):
        crawl = self.crawl(self.base_url,self.depth,0,set())
        check_domain =  self.check_domain(self.base_url,crawl) 
        check_not_domain = self.check_not_domain(self.base_url,crawl)
        check_ref = self.check_ref(check_not_domain,self.target_links)
        return check_domain,check_ref
    
    def crawl(self,url,n, depth,visited):
        if depth < n :
            visited.add(url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            time.sleep(0.3)
            response = requests.get(url,headers=headers)
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                soup = BeautifulSoup(response.text, 'lxml')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    link = link.replace(' ','')
                    visited.add(link)
                    if link.startswith(url):
                        self.crawl(link,n=n,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(self,base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result
    
    def check_not_domain(self,base_url,links):
        result= set()
        for link in links :
            if not link.startswith(base_url):
                result.add(link)
        return result
    
    def check_ref(self,links,target_links):
        for i in links:
            for j in target_links:
                if i.startswith(j):
                    target_links[j]+=1
        return target_links

In [182]:
target_links = ['http://www.bbc.com']
for i in target_links:
    web_spyder=spyder(target_links,i,2)
    link=web_spyder.get_crawler()
print(link)

{'http://www.bbc.com/marathi/articles/cv29zmyg93go', 'http://www.bbc.com/persian/articles/cy9d53ylxxxo', 'https://www.bbc.co.uk/accessibility', 'http://www.bbc.com/turkce/articles/c4n9dp9dwzpo', 'https://ct.moreover.com/?a=50233707343&p=14e&v=1&x=_8po8zSSjJp9yA-GU5EJvA&u1=ND&u2=up-urn:user:PA187552270', 'http://www.bbc.com/japanese/features-and-analysis-64871140', 'http://www.bbc.com/sinhala/world-64820942', 'http://www.bbc.com/urdu/articles/c3g7j2q34qvo', 'http://www.bbc.com/swahili/articles/c88d493kj1no', 'http://www.bbc.com/azeri/articles/ck7jdnd70gpo', 'https://cleanclothes.org/fashions-problems/waste-and-pollution', 'http://www.bbc.com/news/business-45489065', 'http://www.bbc.com/japanese/64935675', 'http://www.bbc.com/tamil/topics/c2dwqdn01v5t', 'http://www.bbc.com/news/av/uk-64929759', 'http://www.bbc.com/persian/iran-64917146', 'http://www.bbc.com/zhongwen/simp/science-62973314', 'http://www.bbc.com/zhongwen/simp/chinese-news-64873353', 'https://www.facebook.com/bbcnewsbrasil/'

In [ ]:
def spacy_process(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    
#Tokenization and lemmatization 
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    #print("Tokenize+Lemmatize:")
    #print(lemma_list)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    #Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
    #print(" ")
    #3print("Remove stopword & punctuation: ")
    #print(filtered_sentence)
    return filtered_sentence

In [ ]:
def cleansing(body):
    for i in body:
        output = i.replace('\n', '  ').replace('\xa0', '  ').replace('®', ' ').replace(';', ' ')
        output = " ".join(output.split())
    return output

In [ ]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  body_tag = soup.find('body')
  text_below_body = body_tag.get_text() 
  body_list =[]
  body_list.append(text_below_body)
  return (body_list,title_tag)


In [ ]:
def get_word(body):
    words = spacy_process(body)
    word_freq = {}
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
    return word_freq

In [ ]:
def make_doc(link,target_links):
    print(link)
    link.replace(" ", "")
    d=dict()    
    body, title = scrape_tags(link)
    body=cleansing(body)
    word = get_word(body)
    d['link']= link
    d['title'] = title
    d['body']=body
    d['location']='location'
    d['word'] = word
    print(d['link'])
    for k in target_links:
        if link.startswith(k):
            d['ref'] = target_links[k]
    return d


In [181]:
link = ['http://www.bbc.com']
for i in link:
    make_doc(i,link)

http://www.bbc.com
http://www.bbc.com


TypeError: list indices must be integers or slices, not str

In [ ]:
def get_doc(target_links,n):
    doc=[]
    num=0
    for i in target_links:
        print(target_links,i,n)
        web_spyder=spyder(target_links,i,n)
        domain_links,target_links =web_spyder.get_all()
        print('all link =', len(domain_links))
        for j in domain_links:
            num+=1
            d = make_doc(j,target_links)
            doc.append(d)
            print(num)
    return doc

In [ ]:
def update_tf_idf():
    conn = sqlite3.connect('../Week10/inverted_index2.db',timeout=3)

    cursor = conn.execute('SELECT COUNT(*) FROM documents')
    N = cursor.fetchone()[0]
    
    cursor = conn.execute('SELECT ID, Word FROM words')
    words = cursor.fetchall()
    
    for word in words:
        word_id = word[0]
        word_str = word[1]

        cursor = conn.execute('SELECT Doc_ID, Frequency FROM word_frequencies WHERE Word_ID = ?', (word_id,))
        doc_freqs = cursor.fetchall()

        df = len(doc_freqs)
        idf = math.log(N / df)

        for doc_freq in doc_freqs:
            doc_id = doc_freq[0]
            tf = doc_freq[1]
            tf_idf = tf * idf
            conn.execute('UPDATE word_frequencies SET TF_IDF = ? WHERE Word_ID = ? AND Doc_ID = ?', (tf_idf, word_id, doc_id))

    conn.commit()



In [ ]:
def insert_to_database(doc):
  conn = sqlite3.connect('../Week10/inverted_index2.db')
  for i in doc:
    conn.execute('''INSERT INTO documents (Link, Title, Body, Location, Ref,Time) VALUES (?, ?, ?, ?, ?, ?);''', (i['link'], i['title'], i['body'],i['location'],i['ref'],datetime.now()))
    doc_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
    
    for j in i['word'].keys():
      word_id = conn.execute("SELECT id FROM words WHERE word = ?", (j,)).fetchone()
      if not word_id:
        conn.execute("INSERT INTO words (word) VALUES (?)", (j,))
        word_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
      else:
        word_id = word_id[0]
      
      conn.execute('''INSERT INTO word_frequencies (word_id, doc_id, Frequency) VALUES (?, ?, ?);''', (word_id, doc_id, i['word'][j]))
  
 
    
  conn.commit()
  update_tf_idf()


In [ ]:
def delete_data(link):
    conn = sqlite3.connect('../Week10/inverted_index2.db',timeout=10)
    doc_id = conn.execute('''
    SELECT id FROM documents WHERE link = ?; ''', (link,)).fetchone()[0]
    conn.execute('''
        DELETE FROM documents WHERE link = ?; ''', (link,))

    conn.execute('''
        DELETE FROM word_frequencies WHERE Doc_ID = ?;''', (doc_id,))

    conn.execute('''
        DELETE FROM words
        WHERE NOT EXISTS (SELECT 1 FROM word_frequencies WHERE word_frequencies.word_id = words.id );''')
    
    conn.commit()
    update_tf_idf()


In [174]:
def update_data(link,n):
    target_links={}
    target_links[link]=0
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    for i in target_links:
        get_link = spyder(target_links,i,n)
        domain_link,target_links = get_link.get_all()
    for j in domain_link:
        link = conn.execute('''SELECT  documents.link
                                    FROM documents
                                    WHERE documents.link = ?
                                    ''',(j,)) 
        link = link.fetchone()
        doc = [make_doc(j,target_links)]
        if link == None :
            print(j)
            insert_to_database(doc)
        else:
            print(j)
            delete_data(j)
            insert_to_database(doc)
    conn.close()

In [176]:
update_data('https://www.bbc.com/news',3)

https://www.bbc.com/news/world-africa-64913796
https://www.bbc.com/news/world-africa-64913796
https://www.bbc.com/news/world-africa-64913796
https://www.bbc.com/news/world-64914542
https://www.bbc.com/news/world-64914542
https://www.bbc.com/news/world-64914542
https://www.bbc.com/news/av/entertainment-arts-64894710
https://www.bbc.com/news/av/entertainment-arts-64894710
https://www.bbc.com/news/av/entertainment-arts-64894710
https://www.bbc.com/news/world-60525350
https://www.bbc.com/news/world-60525350
https://www.bbc.com/news/world-60525350
https://www.bbc.com/news/world-europe-64880268
https://www.bbc.com/news/world-europe-64880268
https://www.bbc.com/news/world-europe-64880268
https://www.bbc.com/news/politics/parliaments
https://www.bbc.com/news/politics/parliaments
https://www.bbc.com/news/politics/parliaments
https://www.bbc.com/news/entertainment_and_arts
https://www.bbc.com/news/entertainment_and_arts
https://www.bbc.com/news/entertainment_and_arts
https://www.bbc.com/news/pol

KeyboardInterrupt: 

In [ ]:
sentence = [str(input())]
clean_sentence = cleansing(sentence)
word = spacy_process(clean_sentence)
print(clean_sentence)
print(word)

In [ ]:
def search():
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    cursor = conn.cursor()
    search_term = [str(input())]
    clean_sentence = cleansing(search_term)
    words = spacy_process(clean_sentence)
    doc_lists = []
    for word in words:
        cursor.execute('SELECT Doc_ID FROM word_frequencies WHERE Word_ID = (SELECT ID FROM words WHERE Word = ?)', (word,))
        doc_lists.append(set(doc[0] for doc in cursor.fetchall()))

    # Find the documents that contain all the words in the sentence
    matching_docs = set.intersection(*doc_lists)

    # Calculate the score for each document
    scores = {}
    for doc_id in matching_docs:
        score = sum(cursor.execute('SELECT Frequency FROM word_frequencies WHERE Word_ID = (SELECT ID FROM words WHERE Word = ?) AND Doc_ID = ?', (word, doc_id)).fetchone()[0] for word in words)
        scores[doc_id] = score

    # Sort the documents by score in descending order
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Get the list of document links
    doc_links = []
    for doc_id, score in sorted_docs:
        cursor.execute('SELECT Link, title FROM documents WHERE ID = ?', (doc_id,))
        doc_links.append(cursor.fetchone()[0])
        
    
            

    # Close the database connection and return the list of document links
    conn.close()
    print(search_term)
    return doc_links

In [ ]:
search()

In [160]:
import sqlite3
import math

def search_sentence():
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    cursor = conn.cursor()
    search_term = str(input())
    list_term = [search_term]
    clean_sentence = cleansing(list_term)
    words = spacy_process(clean_sentence)

    # Retrieve the word IDs for each word in the sentence
    word_ids = []
    for word in words:
        cursor.execute('SELECT word_ID FROM words WHERE word = ?', (word,))
        result = cursor.fetchone()
        if result:
            word_ids.append(result[0])

    # Join the word_frequencies and documents tables to get the documents that contain any of the words in the sentence
    cursor.execute('''
        SELECT documents.Link, documents.Title, SUM(word_frequencies.TF_IDF)
        FROM word_frequencies
        JOIN documents ON documents.ID = word_frequencies.Doc_ID
        WHERE word_frequencies.Word_ID IN ({})
        GROUP BY documents.ID
        ORDER BY SUM(word_frequencies.TF_IDF) DESC
    '''.format(','.join(str(id) for id in word_ids)))

    # Get the document links and titles for the top results
    results = cursor.fetchall()
    print('search term : ',search_term)
    print('search result: ')
    for result in results:
        print(result[0], result[1])


In [ ]:
search_sentence()

In [161]:
import sqlite3
import math

def sentence_search():
    conn = sqlite3.connect('../Week10/inverted_index2.db')
    cursor = conn.cursor()

    # Split the query into individual words
    search_term = [str(input())]
    clean_sentence = cleansing(search_term)
    words = spacy_process(clean_sentence)

    # Retrieve the documents that contain each word
    doc_lists = []
    for word in words:
        cursor.execute("SELECT Doc_ID, TF_IDF FROM word_frequencies JOIN words ON words.ID = word_frequencies.word_ID WHERE word = ?", (word,))
        doc_list = cursor.fetchall()
        doc_lists.append(doc_list)

    # Merge the document lists using the TF-IDF scores
    doc_scores = {}
    for doc_list in doc_lists:
        for doc_id, tf_idf in doc_list:
            if doc_id in doc_scores:
                doc_scores[doc_id] += tf_idf
            else:
                doc_scores[doc_id] = tf_idf

    # Rank the documents by their overall relevance
    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

    # Retrieve the links and titles of the top documents
    results = []
    for doc_id, score in ranked_docs:
        cursor.execute("SELECT Link, Title FROM documents WHERE ID = ?", (doc_id,))
        link, title = cursor.fetchone()
        results.append((link, title))

    conn.close()

    return results


In [173]:
a =sentence_search()
for i in a :
    print (i)

('https://www.bbc.com/news/world-europe-64905649', 'Turkey earthquake: Istanbul residents fear homes will collapse - BBC News')
('https://www.bbc.com/news/topics/c5y9wr365gmt', 'Space debris - BBC News')
('https://www.bbc.com/news/world-africa-64862607', "Pictures from above: What a bird's eye view of Africa reveals - BBC News")
('https://www.bbc.com/news/world/europe', 'Europe - BBC News')
('https://www.bbc.com/news/topics/cx1m7zg0w5zt', 'Syria - BBC News')
('https://www.bbc.com/news/world', 'World - BBC News')
('https://www.bbc.com/news/topics/cq0zxdd0y39t', 'Turkeyâ\x80\x93Syria earthquakes 2023 - BBC News')
('https://www.bbc.com/news/world-europe-64824516', 'Turkey earthquake: Survivors living in fear on streets - BBC News')
('https://www.bbc.com/news/topics/cvenzmgyww4t', 'Space exploration - BBC News')
('https://www.bbc.com/news/world-asia-64871603', 'Japan forced to destroy flagship H3 rocket in failed launch - BBC News')
('https://www.bbc.com/news/world-europe-61003878', "Bucha